In [6]:
import cohere
import numpy as np
import pandas as pd
import streamlit as st
from annoy import AnnoyIndex

# Access the API key value
#api_key = st.secrets['API_KEY']
api_key = '4jdEqGb3coPXaw7M8mbEaTvKMdYu5vJsa0G3MHbL'

In [7]:
df = pd.read_csv('cohere_text_preprocessing.csv')
df

,text,link,title,Category,Type
0,"[""Cohere For AI launched in June of this year ...",https://txt.cohere.ai/c4ai-2022/,"Exploring the Unknown, Together: 2022 at Coher...",Research,Blog
1,"[""Since Cohereâ€™s beginning, we've been focus...",https://txt.cohere.ai/martin-kon/,Cohere Welcomes Martin Kon as President & COO,Press,Blog
2,"['Humans speak over 71001 languages, yet the m...",https://txt.cohere.ai/multilingual/,Cohere's Multilingual Text Understanding Model...,Product Launch,Blog
3,"['In this multi-part guide, we will go through...",https://txt.cohere.ai/generative-ai-part-1/,Generative AI with Cohere: Part 1,Guide,Blog
4,['- This roundup highlights some interesting N...,https://txt.cohere.ai/top-nlp-papers-november-...,Top NLP Papers—November 2022,Research,Blog
...,...,...,...,...,...
296,"Build trust with your users, and the general p...",https://cohere.ai/use-case-toxic-language,Toxic Language,Use Cases,Product Documentation
297,"Supercharge your moderation efforts, Content m...",https://cohere.ai/use-case-toxic-language,Toxic Language,Use Cases,Product Documentation
298,"Social media platforms, Cohere’s large languag...",https://cohere.ai/use-case-toxic-language,Toxic Language,Use Cases,Product Documentation
299,"Gaming platforms, most gamers want better solu...",https://cohere.ai/use-case-toxic-language,Toxic Language,Use Cases,Product Documentation


In [8]:
# add an id column
df['id'] = df.index

In [9]:
def chunk_text(df, width=1500, overlap=500):
    # create an empty dataframe to store the chunked text
    new_df = pd.DataFrame(columns=['id', 'text_chunk'])

    # iterate over each row in the original dataframe
    for index, row in df.iterrows():
        # split the text into chunks of size 'width', with overlap of 'overlap'
        chunks = []
        rows = []
        for i in range(0, len(row['text']), width - overlap):
            chunk = row['text'][i:i+width]
            chunks.append(chunk)

        # iterate over each chunk and add it to the new dataframe
        chunk_rows = []
        for i, chunk in enumerate(chunks):
            # calculate the start index based on the chunk index and overlap
            start_index = i * (width - overlap)

            # create a new row with the chunked text and the original row's ID
            new_row = {'id': row['id'], 'text_chunk': chunk, 'start_index': start_index}
            chunk_rows.append(new_row)
        chunk_df = pd.DataFrame(chunk_rows)
        new_df = pd.concat([new_df, chunk_df], ignore_index=True)

    return new_df



new_df = chunk_text(df)
# append text chunks to the original dataframe in id order
df = df.merge(new_df, on='id', how='left')
df.Category

0       Research
1       Research
2       Research
3       Research
4       Research
         ...    
540    Use Cases
541    Use Cases
542    Use Cases
543    Use Cases
544    Use Cases
Name: Category, Length: 545, dtype: object

In [11]:
co = cohere.Client(api_key) 

# Get the embeddings
embeds = co.embed(texts=list(df['text_chunk']),
                  model="embed-english-v3.0",
                  truncate="RIGHT").embeddings
# Check the dimensions of the embeddings
embeds = np.array(embeds)
embeds.shape


(545, 4096)

In [12]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('search_index.ann')

True

In [13]:
# export the dataframe to a csv file
df.to_csv('cohere_text_final.csv', index=False)